In [2]:
import os
import scipy.io
import numpy as np

import xgboost as xgb
#import lightgbm as lgb
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostClassifier

from sklearn.model_selection import train_test_split #to split the dataset for training and testing
from sklearn.svm import SVC   #for Support Vector Machine (SVM) Algorithm

from sklearn import preprocessing
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

from sklearn.neighbors import KNeighborsClassifier  # for K nearest neighbours
from sklearn.tree import DecisionTreeClassifier #for using Decision Tree Algoithm


import tensorflow as tf
from keras.utils.vis_utils import plot_model
from keras.preprocessing.sequence import TimeseriesGenerator

from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Conv1D, Conv2D, MaxPooling1D

from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt


2024-11-26 15:05:37.022213: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
import os
import scipy.io
import numpy as np

def load_data(folder_path, exclude_filetypes=[]):
    all_epochs = []
    all_triggers = []
    all_labels = []
    all_stimuli = []
    all_phase = []
    all_subjects = []

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".mat"):
                filepath = os.path.join(root, file)
                print(file)
                
                # Check if the file should be excluded based on the exclude_filetypes list
                if any(filetype in file for filetype in exclude_filetypes):
                    print(f'Excluding file types {file}')
                    continue

                mat = scipy.io.loadmat(filepath)
                # Replace 'epoch' with the actual key in your .mat files
                data = mat['epoch']  # Shape: (n_channels, n_times, n_epochs)
                
                if data is None:
                    print(f'File {filepath} is empty')
                    continue
                print(data.shape)
                if len(data.shape) < 3:
                    data = np.expand_dims(data,axis=2)
                # Get the trigger type from the filename
                trigger_type = file.split('_type')[-1].split('.')[0]
                trigger_type = int(trigger_type.split('__')[0].split('.')[0])
                print(trigger_type)
                # Create a list of triggers based on the number of epochs
                triggers = [trigger_type] * data.shape[2]
                
                # Get the label from the folder name
                label = 0 if 'Not_Blinded' in root else 1
                
                # Append data to the lists
                all_epochs.append(data)
                all_triggers.extend(triggers)
                all_labels.extend([label] * data.shape[2])
                #stimulus, phase = get_stimulus_and_phase(filepath) # NORMAL IB
                #stimulus, phase = get_stimulus_and_phase_oddball(filepath) # ODDBALL
                stimulus, phase = get_stimulus_and_phase(filepath) # ODDBALL
                all_stimuli.extend([stimulus] * data.shape[2])
                all_phase.extend([phase] * data.shape[2])
                all_subjects.extend([get_subject_number(filepath)] * data.shape[2])
                
    # Concatenate lists into numpy arrays
    all_epochs = np.concatenate(all_epochs, axis=2)  # Shape: (n_channels, n_times, total_epochs)
    all_triggers = np.array(all_triggers)  # Shape: (total_epochs,)
    all_labels = np.array(all_labels)  # Shape: (total_epochs,)

    return all_epochs, all_triggers, all_labels, all_phase, all_stimuli, all_subjects

# Assuming get_stimulus_and_phase and get_subject_number are defined elsewhere


In [5]:
import re

def get_subject_number(file_name):
    match = re.search(r'IB_Sub_(\d+)_', file_name)
    if match:
        #print(int(match.group(1)))
        return int(match.group(1))
        
    else:
        #print("Found None Subj Num")
        return None  # or handle the case where no subject number is found
        
        #print(int(match.group(1)))


In [6]:
def get_stimulus_and_phase_ERN(file_name):
    stimulus = ""
    phase = ""

# '12180','13180','22180','23180'
    if '12180' in file_name:
        stimulus = 'correct'
        phase = 'invisible'
    elif '13180' in file_name:
        stimulus = 'error'
        phase = 'invisible'
    elif '22180' in file_name:
        stimulus = 'correct'
        phase = 'visible'
    elif '23180' in file_name:
        stimulus = 'error'
        phase = 'visible'
        
    return stimulus, phase

# Example usage
#stimulus, phase = get_stimulus_and_phase(file_name)
#
# 
# 11132

In [7]:
def get_stimulus_and_phase_oddball(file_name):
    stimulus = ""
    phase = ""

    if '11132' in file_name:
        stimulus = 'same'
        phase = 'invisible'
    elif '11133' in file_name:
        stimulus = 'different'
        phase = 'invisible'
    elif '21132' in file_name:
        stimulus = 'same'
        phase = 'visible'
    elif '21133' in file_name:
        stimulus = 'different'
        phase = 'visible'
    elif '31132' in file_name:
        stimulus = 'same'
        phase = 'reported'
    elif '31133' in file_name:
        stimulus = 'different'
        phase = 'reported'
        
    elif '12132' in file_name:
        stimulus = 'error_same'
        phase = 'invisible'
    elif '12133' in file_name:
        stimulus = 'error_different'
        phase = 'invisible'
    elif '22132' in file_name:
        stimulus = 'error_same'
        phase = 'visible'
    elif '22133' in file_name:
        stimulus = 'error_different'
        phase = 'visible'
    elif '32132' in file_name:
        stimulus = 'error_same'
        phase = 'reported'
    elif '32133' in file_name:
        stimulus = 'error_different'
        phase = 'reported'
        
    #print(f"Stimulus: {stimulus}, Phase: {phase}")

    return stimulus, phase

# Example usage
#stimulus, phase = get_stimulus_and_phase(file_name)
#
# 
# 11132

In [8]:
def get_stimulus_and_phase(file_name):
    stimulus = ""
    phase = ""

    if 'type210' in file_name:
        stimulus = 'face'
        phase = 'invisible'
    elif 'type220' in file_name:
        stimulus = 'face'
        phase = 'visible'
    elif 'type230' in file_name:
        stimulus = 'face'
        phase = 'reported'
    elif 'type211' in file_name:
        stimulus = 'house'
        phase = 'invisible'
    elif 'type221' in file_name:
        stimulus = 'house'
        phase = 'visible'
    elif 'type231' in file_name:
        stimulus = 'house'
        phase = 'reported'
    elif 'type212' in file_name:
        stimulus = 'noise'
        phase = 'invisible'
    elif 'type222' in file_name:
        stimulus = 'noise'
        phase = 'visible'
    elif 'type232' in file_name:
        stimulus = 'noise'
        phase = 'reported'
        
    #print(f"Stimulus: {stimulus}, Phase: {phase}")

    return stimulus, phase

# Example usage
#stimulus, phase = get_stimulus_and_phase(file_name)
#


In [13]:
blinded_folder = "/Users/ugobruzadinnunes/Desktop/FIN/Epochs_full/BLINDED"

not_blinded_folder = "/Users/ugobruzadinnunes/Desktop/FIN/Epochs_full/NOTBLINDED"

In [10]:

# Load data from both folders
blinded_folder = "D:\\Triangulation\\Reed\\Raw\\Subjects\\DONE\\DONE_ICA\\DONE_ICA\\DONE_ICA\\Epochs_full"
not_blinded_folder = "D:\\Triangulation\\Reed\\Raw\\Subjects\\DONE\\DONE_ICA\\DONE_ICA\\DONE_ICA\\Epochs_full"

blinded_folder = "D:\\Triangulation\\Reed\\Raw\\Subjects\\DONE\\DONE_ICA\\DONE_ICA\\DONE_ICA\\Epochs_full"

disk_folder = "D:\\Triangulation\\Reed\\Raw\\Subjects\\DONE\\DONE_ICA\\DONE_ICA\\DONE_ICA\\Disk_Trigger_added\\Epochs_full\\Blinded"

ERN_folder = "D:\\Triangulation\\Reed\\Raw\\ERN\\Epochs_full\\Epochs_full"


In [21]:
epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b = load_data(blinded_folder)

IB_Sub_33_mergedsICAPJ3sPJ1sPJ1sPJ1sPJ1sTJ40_type211__Blinded.mat
(64, 750, 183)
IB_Sub_103_500hz_HM64_Hp1Lp55NF60_Ep_ICA10_IC_rej1_Ch_rej35_50_type222__Blinded.mat
(64, 750, 192)
IB_Sub_32_mergedsICAPJ4sPJ1sTJ40_type231__Blinded.mat
(64, 750, 208)
IB_Sub_61_500hz_HM64_Hp1Lp55NF60_Ep_ICA06_Ic_rej1_3sPJ3sTJ3_type220__Blinded.mat
(64, 750, 192)
IB_Sub_20_500hz_HM64_Hp1Lp55NF60_Ep_ICA07_Ic_rej1_type231__Blinded.mat
(64, 750, 192)
IB_Sub_66_500hz_HM64_Hp1Lp55NF60_Ep_ICA06_Ic_rej1_2sPJ2sPJ1sTJ16sTJ6_type210__Blinded.mat
(64, 750, 189)
IB_Sub_51_500hz_HM64_Hp1Lp55NF60_Ep_ICA01_Ic_rej1_2_3_5_flagsPJ10sPJ1sPJ1_type212__Blinded.mat
(64, 750, 192)
IB_Sub_76_500hz_HM64_Hp1Lp55NF60_Ep_ICA06_Co_Rej1_Ic_rej1_6_flagsPJ1sPJ2sPJ1sTJ90_type230__Blinded.mat
(64, 750, 174)
IB_Sub_5_500hz_HM64_Hp1Lp55NF60_Ep_ICA11_Ic_rej1sTJ2sPJ1125mvDONE_type222__Blinded.mat
(64, 750, 192)
IB_Sub_60_500hz_HM64_Hp1Lp55NF60_Ep_ICA09_Ic_rej2_3_4sPJ8sPJ1_type221__Blinded.mat
(64, 750, 192)
IB_Sub_112_500hz_HM64_Hp1Lp55NF60_Ep

In [22]:
import pickle
with open('IB_eeg_data_final_BLINDED.pkl', 'wb') as f:
    pickle.dump((epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b), f)

In [ ]:
epochs_nblinded, triggers_nblinded, labels_nblinded, phase_nb, stimuli_nb, subjects_nb = load_data(not_blinded_folder)

IB_Sub_3_500hz_HM64_Hp1Lp55NF60_Ep_ICA12_Ic_rej1_6_7_8sPJ1sPJ1sTJ24_type221__NotBlinded.mat
(64, 750, 190)
221
IB_Sub_57_500hz_HM64_Hp1Lp55NF60_Ep_ICA10_Ic_rej1sTJ4mv125DONE_type231__NotBlinded.mat
(64, 750, 192)
231
IB_Sub_89_500hz_HM64_Hp1Lp55NF60_Ep_ICA06_Ic_rej!_to_6_FLAGsCI37sCI33sPJ3sTJ11_type212__NotBlinded.mat
(64, 750, 191)
212
IB_Sub_21_500hz_HM64_Hp1Lp55NF60_Ep_ICA09_Ic_rej1_2_4_5sTJ41_type221__NotBlinded.mat
(64, 750, 189)
221
IB_Sub_26_500hz_HM64_Hp1Lp55NF60_Ep_ICA16_Ic_rej1_2sTJ22sICAPJ3TJ1needsMinMax_type222__NotBlinded.mat
(64, 750, 192)
222
IB_Sub_67_500hz_HM64_Hp1Lp55NF60_Ep_ICA09_Ic_rej1_2sPJ2sTJ4sPJ2sTJ9125mv_type230__NotBlinded.mat
(64, 750, 190)
230
IB_Sub_27_500hz_HM64_Hp1Lp55NF60_Ep_ICA02_Ic_rej1_2_3_4_FLAGsTJ174weirdslowearly_type210__NotBlinded.mat
(64, 750, 147)
210
IB_Sub_101_500hz_HM64_Hp1Lp55NF60_Ep_ICA07_IC_rej1_DONE_type220__NotBlinded.mat
(64, 750, 192)
220
IB_Sub_10_500hz_HM64_Hp1Lp55NF60_Ep_ICA11_IC_rej1sPJ2sTJ34sPJ1sTJ19DONE_type222__NotBlinded.mat
(

In [ ]:
import pickle
with open('IB_eeg_data_final_NOTBLINDED.pkl', 'wb') as f:
    pickle.dump((epochs_nblinded, triggers_nblinded, labels_nblinded, phase_nb, stimuli_nb, subjects_nb), f)

In [34]:
# DISK DATA SAVE
epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b = load_data(disk_folder)

IB_Sub_11_500hz_HM63_Hp1Lp55NF60_LEfix_Ep_ICA_IcRj04_EpRj0006DONE_Disc_type11132.mat
(64, 750, 184)
IB_Sub_11_500hz_HM63_Hp1Lp55NF60_LEfix_Ep_ICA_IcRj04_EpRj0006DONE_Disc_type11133.mat
(64, 750, 186)
IB_Sub_11_500hz_HM63_Hp1Lp55NF60_LEfix_Ep_ICA_IcRj04_EpRj0006DONE_Disc_type12132.mat
(64, 750, 9)
IB_Sub_11_500hz_HM63_Hp1Lp55NF60_LEfix_Ep_ICA_IcRj04_EpRj0006DONE_Disc_type12133.mat
(64, 750, 7)
IB_Sub_11_500hz_HM63_Hp1Lp55NF60_LEfix_Ep_ICA_IcRj04_EpRj0006DONE_Disc_type21132.mat
(64, 750, 196)
IB_Sub_11_500hz_HM63_Hp1Lp55NF60_LEfix_Ep_ICA_IcRj04_EpRj0006DONE_Disc_type21133.mat
(64, 750, 170)
IB_Sub_11_500hz_HM63_Hp1Lp55NF60_LEfix_Ep_ICA_IcRj04_EpRj0006DONE_Disc_type22132.mat
(64, 750, 8)
IB_Sub_11_500hz_HM63_Hp1Lp55NF60_LEfix_Ep_ICA_IcRj04_EpRj0006DONE_Disc_type22133.mat
(64, 750, 14)
IB_Sub_11_500hz_HM63_Hp1Lp55NF60_LEfix_Ep_ICA_IcRj04_EpRj0006DONE_Disc_type31132.mat
(64, 750, 183)
IB_Sub_11_500hz_HM63_Hp1Lp55NF60_LEfix_Ep_ICA_IcRj04_EpRj0006DONE_Disc_type31133.mat
(64, 750, 196)
IB_Sub_

In [36]:
import pickle
with open('IB_eeg_data_DISK_BLINDED.pkl', 'wb') as f:
    pickle.dump((epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b), f)

In [1]:

epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b = load_data(blinded_folder,['type22','type23'])


NameError: name 'load_data' is not defined

In [14]:
import pickle

epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b = load_data(blinded_folder,['type22','type23'])

# Save the data
with open('IB_eeg_data_final_BLINDED_phase1.pkl', 'wb') as f:
    pickle.dump((epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b), f)

IB_Sub_33_mergedsICAPJ3sPJ1sPJ1sPJ1sPJ1sTJ40_type211__Blinded.mat
(64, 750, 183)
211
IB_Sub_103_500hz_HM64_Hp1Lp55NF60_Ep_ICA10_IC_rej1_Ch_rej35_50_type222__Blinded.mat
Excluding file types IB_Sub_103_500hz_HM64_Hp1Lp55NF60_Ep_ICA10_IC_rej1_Ch_rej35_50_type222__Blinded.mat
IB_Sub_32_mergedsICAPJ4sPJ1sTJ40_type231__Blinded.mat
Excluding file types IB_Sub_32_mergedsICAPJ4sPJ1sTJ40_type231__Blinded.mat
IB_Sub_61_500hz_HM64_Hp1Lp55NF60_Ep_ICA06_Ic_rej1_3sPJ3sTJ3_type220__Blinded.mat
Excluding file types IB_Sub_61_500hz_HM64_Hp1Lp55NF60_Ep_ICA06_Ic_rej1_3sPJ3sTJ3_type220__Blinded.mat
IB_Sub_20_500hz_HM64_Hp1Lp55NF60_Ep_ICA07_Ic_rej1_type231__Blinded.mat
Excluding file types IB_Sub_20_500hz_HM64_Hp1Lp55NF60_Ep_ICA07_Ic_rej1_type231__Blinded.mat
IB_Sub_66_500hz_HM64_Hp1Lp55NF60_Ep_ICA06_Ic_rej1_2sPJ2sPJ1sTJ16sTJ6_type210__Blinded.mat
(64, 750, 189)
210
IB_Sub_51_500hz_HM64_Hp1Lp55NF60_Ep_ICA01_Ic_rej1_2_3_5_flagsPJ10sPJ1sPJ1_type212__Blinded.mat
(64, 750, 192)
212
IB_Sub_76_500hz_HM64_Hp1Lp55

In [15]:
epochs_blinded = []
epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b = load_data(blinded_folder,['type21','type23'])
import pickle
# Save the data
with open('IB_eeg_data_BLINDED_phase2.pkl', 'wb') as f:
    pickle.dump((epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b), f)

IB_Sub_33_mergedsICAPJ3sPJ1sPJ1sPJ1sPJ1sTJ40_type211__Blinded.mat
Excluding file types IB_Sub_33_mergedsICAPJ3sPJ1sPJ1sPJ1sPJ1sTJ40_type211__Blinded.mat
IB_Sub_103_500hz_HM64_Hp1Lp55NF60_Ep_ICA10_IC_rej1_Ch_rej35_50_type222__Blinded.mat
(64, 750, 192)
222
IB_Sub_32_mergedsICAPJ4sPJ1sTJ40_type231__Blinded.mat
Excluding file types IB_Sub_32_mergedsICAPJ4sPJ1sTJ40_type231__Blinded.mat
IB_Sub_61_500hz_HM64_Hp1Lp55NF60_Ep_ICA06_Ic_rej1_3sPJ3sTJ3_type220__Blinded.mat
(64, 750, 192)
220
IB_Sub_20_500hz_HM64_Hp1Lp55NF60_Ep_ICA07_Ic_rej1_type231__Blinded.mat
Excluding file types IB_Sub_20_500hz_HM64_Hp1Lp55NF60_Ep_ICA07_Ic_rej1_type231__Blinded.mat
IB_Sub_66_500hz_HM64_Hp1Lp55NF60_Ep_ICA06_Ic_rej1_2sPJ2sPJ1sTJ16sTJ6_type210__Blinded.mat
Excluding file types IB_Sub_66_500hz_HM64_Hp1Lp55NF60_Ep_ICA06_Ic_rej1_2sPJ2sPJ1sTJ16sTJ6_type210__Blinded.mat
IB_Sub_51_500hz_HM64_Hp1Lp55NF60_Ep_ICA01_Ic_rej1_2_3_5_flagsPJ10sPJ1sPJ1_type212__Blinded.mat
Excluding file types IB_Sub_51_500hz_HM64_Hp1Lp55NF60_Ep

In [13]:
epochs_blinded = []
epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b = load_data(blinded_folder,['type21','type22'])
import pickle
# Save the data
with open('IB_eeg_data_BLINDED_phase3.pkl', 'wb') as f:
    pickle.dump((epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b), f)

In [9]:
epochs_blinded = []
epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b = load_data(not_blinded_folder,['type22','type23'])
import pickle
# Save the data
with open('IB_eeg_data_NOTBLINDED_phase1.pkl', 'wb') as f:
    pickle.dump((epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b), f)

In [11]:
epochs_blinded = []
epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b = load_data(not_blinded_folder,['type21','type23'])
import pickle
# Save the data
with open('IB_eeg_data_NOTBLINDED_phase2.pkl', 'wb') as f:
    pickle.dump((epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b), f)

In [12]:
epochs_blinded = []
epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b = load_data(not_blinded_folder,['type21','type22'])
import pickle
# Save the data
with open('IB_eeg_data_NOTBLINDED_phase3.pkl', 'wb') as f:
    pickle.dump((epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b), f)

In [10]:
epochs_blinded = []

In [11]:
epochs_not_blinded, triggers_not_blinded, labels_not_blinded, phase_n, stimuli_n, subjects_n  = load_data(not_blinded_folder)

In [12]:
    # Save the data
with open('IB_eeg_data_NEW.pkl', 'wb') as f:
    pickle.dump((epochs_not_blinded, triggers_not_blinded, labels_not_blinded, phase_n, stimuli_n, subjects_n), f)

In [8]:

epochs_not_blinded, triggers_not_blinded, labels_not_blinded, phase_n, stimuli_n, subjects_n  = load_data(not_blinded_folder)

# Combine data from both folders
epochs = np.concatenate([epochs_blinded, epochs_not_blinded], axis=2)
triggers = np.concatenate([triggers_blinded, triggers_not_blinded])
labels = np.concatenate([labels_blinded, labels_not_blinded])
phase = np.concatenate([phase_b, phase_n])
stim = np.concatenate([stimuli_b, stimuli_n])
subj = np.concatenate([subjects_b, subjects_n])

print("Epochs shape:", epochs.shape)
print("Triggers shape:", triggers.shape)
print("Labels shape:", labels.shape)

print("Epochs shape:", stim.shape)
print("Triggers shape:", phase.shape)
print("Labels shape:", subj.shape)

import pickle


# Save the data
with open('IB_eeg_data_NEW.pkl', 'wb') as f:
    pickle.dump((epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b), f)

# Save the data
with open('IB_eeg_data_NEW.pkl', 'wb') as f:
    pickle.dump((epochs_not_blinded, triggers_not_blinded, labels_not_blinded, phase_n, stimuli_n, subjects_n), f)

# Save the data
# with open('IB_eeg_data_NEW.pkl', 'wb') as f:
#     pickle.dump((epochs, triggers, labels, phase, stim, subj), f)


Epochs shape: (64, 1250, 291272)
Triggers shape: (291272,)
Labels shape: (291272,)
Epochs shape: (291272,)
Triggers shape: (291272,)
Labels shape: (291272,)


MemoryError: 

In [10]:

# # Combine data from both folders
# epochs = np.concatenate([epochs_blinded, epochs_not_blinded], axis=2)
# triggers = np.concatenate([triggers_blinded, triggers_not_blinded])
# labels = np.concatenate([labels_blinded, labels_not_blinded])
# phase = np.concatenate([phase_b, phase_n])
# stim = np.concatenate([stimuli_b, stimuli_n])
# subj = np.concatenate([subjects_b, subjects_n])

# print("Epochs shape:", epochs.shape)
# print("Triggers shape:", triggers.shape)
# print("Labels shape:", labels.shape)

# print("Epochs shape:", stim.shape)
# print("Triggers shape:", phase.shape)
# print("Labels shape:", subj.shape)

# import pickle

# Save the data
with open('IB_eeg_data_NEW.pkl', 'wb') as f:
    pickle.dump((epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b), f)

# Save the data
with open('IB_eeg_data_NEW.pkl', 'wb') as f:
    pickle.dump((epochs_not_blinded, triggers_not_blinded, labels_not_blinded, phase_n, stimuli_n, subjects_n), f)

MemoryError: 